Paso 0: Carga de los datos desde la base SQLite

In [2]:
import sqlite3
import pandas as pd

# Conexión a la base de datos
conn = sqlite3.connect("instacart.db")
cursor = conn.cursor()

# Cargar todas las tablas necesarias
departments = pd.read_sql("SELECT * FROM departments", conn)
aisles = pd.read_sql("SELECT * FROM aisles", conn)
products = pd.read_sql("SELECT * FROM products", conn)
orders = pd.read_sql("SELECT * FROM orders", conn)
order_products_prior = pd.read_sql("SELECT * FROM order_products_prior", conn)
order_products_train = pd.read_sql("SELECT * FROM order_products_train", conn)


PASO 1: Exploración inicial de order_products_prior


In [3]:
# Ver los primeros registros
print(order_products_prior.head())

# Ver estructura general
print(order_products_prior.info())

# Ver estadísticos generales
print(order_products_prior.describe())

# Ver número de duplicados exactos
print("Duplicados:", order_products_prior.duplicated().sum())

# Ver valores únicos por columna
print("Valores únicos por columna:")
print(order_products_prior.nunique())

# Verificar si hay nulos
print("Nulos por columna:")
print(order_products_prior.isnull().sum())


   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB
None
           order_id    product_id  add_to_cart_order     reordered
count  3.243449e+07  3.243449e+07       3.243449e+07  3.243449e+07
mean   1.710749e+06  2.557634e+04       8.351076e+00  5.896975e-01
std    9.873007e+05  1.409669e+04       7.126671e+00  4.918886e-01
min    2.000000e+00  1.000000e+00       1.000000e+00  0.000000e+00
25

In [4]:
primeros_pedidos = orders[orders["order_number"] == 1]
print("¿Coinciden los nulos con el primer pedido?:", primeros_pedidos["days_since_prior_order"].isnull().all())


¿Coinciden los nulos con el primer pedido?: True


PASO 2: Unir order_products_prior con orders para obtener user_id

In [5]:
# Filtrar solo los pedidos previos
orders_prior = orders[orders["eval_set"] == "prior"][["order_id", "user_id"]]

# Unir para añadir user_id a order_products_prior
order_prior_merged = order_products_prior.merge(orders_prior, on="order_id", how="left")

# Verificamos que no haya nulos en user_id
print("¿Nulos en user_id?:", order_prior_merged["user_id"].isnull().sum())

# Mostramos las primeras filas
print(order_prior_merged.head())


¿Nulos en user_id?: 0
   order_id  product_id  add_to_cart_order  reordered  user_id
0         2       33120                  1          1   202279
1         2       28985                  2          1   202279
2         2        9327                  3          0   202279
3         2       45918                  4          1   202279
4         2       30035                  5          0   202279


PASO 3: Crear la matriz user_id × product_id con frecuencia de compra para el modelo SVD

In [6]:
# Agrupar por user_id y product_id para contar cuántas veces ha comprado ese producto
user_product_matrix = order_prior_merged.groupby(["user_id", "product_id"]).size().reset_index(name="purchase_count")

# Mostrar las primeras filas para comprobar
print(user_product_matrix.head())

# Comprobación básica
print("Shape:", user_product_matrix.shape)
print("Nulos:", user_product_matrix.isnull().sum())


   user_id  product_id  purchase_count
0        1         196              10
1        1       10258               9
2        1       10326               1
3        1       12427              10
4        1       13032               3
Shape: (13307953, 3)
Nulos: user_id           0
product_id        0
purchase_count    0
dtype: int64


PASO 4: Preparar los datos para el modelo SVD de Surprise

In [7]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Crear un Reader indicando que los ratings van de 1 en adelante (frecuencia)
reader = Reader(rating_scale=(1, user_product_matrix["purchase_count"].max()))

# Cargar los datos desde el dataframe
data = Dataset.load_from_df(user_product_matrix[["user_id", "product_id", "purchase_count"]], reader)

# Definir modelo SVD
model = SVD()

# Evaluar con validación cruzada
results = cross_validate(model, data, measures=["RMSE", "MAE"], cv=3, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    3.4489  3.4424  3.4482  3.4465  0.0029  
MAE (testset)     1.7598  1.7578  1.7593  1.7590  0.0008  
Fit time          78.47   79.35   78.99   78.94   0.36    
Test time         25.69   24.58   25.81   25.36   0.55    


In [9]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from collections import defaultdict
import numpy as np

# Paso 1: Crear dataset desde el dataframe
reader = Reader(rating_scale=(1, user_product_matrix["purchase_count"].max()))
data = Dataset.load_from_df(user_product_matrix[["user_id", "product_id", "purchase_count"]], reader)

# Paso 2: Dividir en train y test
trainset, testset = train_test_split(data, test_size=0.2)

# Paso 3: Entrenar el modelo en el set de entrenamiento
model = SVD()
model.fit(trainset)

# Paso 4: Hacer predicciones sobre el testset
predictions = model.test(testset)

# Paso 5: Obtener el top-N recomendado para cada usuario
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Ordenar por score estimado descendente y quedarnos con top-N
    for uid, user_ratings in top_n.items():
        top_n[uid] = sorted(user_ratings, key=lambda x: x[1], reverse=True)[:n]
    return top_n

top_n = get_top_n(predictions, n=10)


In [10]:
# Paso 6: Construir diccionarios para comparar
# Lo que el usuario realmente compró en el testset
actual = defaultdict(set)
for uid, iid, true_r in testset:
    if true_r >= 1:  # consideramos que fue "relevante" si se compró al menos una vez
        actual[uid].add(iid)

# Lo que el sistema recomendó
recommended = {uid: set([iid for iid, _ in items]) for uid, items in top_n.items()}

# Paso 7: Métricas agregadas
precision_list, recall_list, f1_list = [], [], []

for uid in actual:
    if uid in recommended:
        true_items = actual[uid]
        rec_items = recommended[uid]
        tp = len(true_items & rec_items)
        fp = len(rec_items - true_items)
        fn = len(true_items - rec_items)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

# Resultado final
print(f"Precision@10: {np.mean(precision_list):.4f}")
print(f"Recall@10:    {np.mean(recall_list):.4f}")
print(f"F1-Score@10:  {np.mean(f1_list):.4f}")


Precision@10: 1.0000
Recall@10:    0.7890
F1-Score@10:  0.8520


In [11]:
user_id = 1  # puedes cambiarlo por cualquier otro que tengas


In [12]:
# Productos que ya compró el usuario (usando la tabla original mergeada)
productos_comprados = order_prior_merged[order_prior_merged["user_id"] == user_id]["product_id"].unique()
print("Ya ha comprado:", len(productos_comprados), "productos.")


Ya ha comprado: 18 productos.


In [13]:
# Todos los product_id posibles
todos_los_productos = products["product_id"].unique()

# Filtramos los que no ha comprado
productos_no_comprados = np.setdiff1d(todos_los_productos, productos_comprados)

# Creamos predicciones para esos productos
predicciones = [model.predict(uid=user_id, iid=pid) for pid in productos_no_comprados]

# Ordenamos por estimación descendente y cogemos los mejores N
top_n_predicciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:10]


In [14]:
# Mostramos los nombres de los productos recomendados
recommended_product_ids = [pred.iid for pred in top_n_predicciones]

# Filtramos de la tabla products
productos_recomendados = products[products["product_id"].isin(recommended_product_ids)]

print("Top-10 recomendaciones para el usuario", user_id)
print(productos_recomendados[["product_id", "product_name"]])


Top-10 recomendaciones para el usuario 1
       product_id                                       product_name
6947         6948                       Organic Original Almond Milk
20297       20298                        Diet Green Tea with Ginseng
22958       22959                 Reduced Fat Milk 100% Lactose Free
35411       35412  Select Tender Chicken with Vegetables & Brown ...
37709       37710                                          Trail Mix
39862       39863                                    Organic 2% Milk
42243       42244  Total 2% All Natural Low Fat 2% Milkfat Greek ...
43393       43394                    Organic Lactose Free Whole Milk
45327       45328                              Flavored Vodka, Peach
48856       48857                           Authentic French Brioche


In [15]:
def recomendar_productos(user_id, model, products_df, order_prior_merged, top_n=10):
    # Productos que ya ha comprado el usuario
    productos_comprados = order_prior_merged[order_prior_merged["user_id"] == user_id]["product_id"].unique()
    todos_los_productos = products_df["product_id"].unique()
    productos_no_comprados = np.setdiff1d(todos_los_productos, productos_comprados)

    # Predecir para los que no ha comprado
    predicciones = [model.predict(uid=user_id, iid=pid) for pid in productos_no_comprados]
    top_n_predicciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:top_n]
    recommended_ids = [pred.iid for pred in top_n_predicciones]

    # Mostrar nombres
    return products_df[products_df["product_id"].isin(recommended_ids)][["product_id", "product_name"]]


In [16]:
recomendar_productos(1, model, products, order_prior_merged, top_n=10)


,product_id,product_name
6947,6948,Organic Original Almond Milk
20297,20298,Diet Green Tea with Ginseng
22958,22959,Reduced Fat Milk 100% Lactose Free
35411,35412,Select Tender Chicken with Vegetables & Brown ...
37709,37710,Trail Mix
39862,39863,Organic 2% Milk
42243,42244,Total 2% All Natural Low Fat 2% Milkfat Greek ...
43393,43394,Organic Lactose Free Whole Milk
45327,45328,"Flavored Vodka, Peach"
48856,48857,Authentic French Brioche


In [18]:
# Filtrar los productos comprados por el usuario 1
compras_usuario = order_prior_merged[order_prior_merged["user_id"] == 1]

# Unir con products para ver los nombres
compras_usuario = compras_usuario.merge(products, on="product_id", how="left")

# Ver productos únicos que ha comprado
productos_comprados = compras_usuario[["product_id", "product_name"]].drop_duplicates().sort_values("product_name")

print(productos_comprados)


    product_id                             product_name
43       38928                 0% Greek Strained Yogurt
33       26088               Aged White Cheddar Popcorn
7        13176                   Bag of Organic Bananas
6        41787                           Bartlett Pears
17       13032                    Cinnamon Toast Crunch
12       30450                     Creamy Almond Butter
5        17122                        Honeycrisp Apples
42       39657                   Milk Chocolate Almonds
4        10326                      Organic Fuji Apples
23       49235                      Organic Half & Half
3        25133                    Organic String Cheese
46       35951          Organic Unsweetened Almond Milk
36       14084  Organic Unsweetened Vanilla Almond Milk
1        12427                      Original Beef Jerky
2        10258                               Pistachios
0          196                                     Soda
22       26405         XL Pick-A-Size Paper Towe

In [17]:
import joblib
joblib.dump(model, "svd_model_good_precision.pkl")


['svd_model_good_precision.pkl']

MBA (MARKET BASKET ANALYSIS)

Paso 1: Filtrar productos frecuentes y muestrear pedidos

In [21]:
# 1. Filtrar productos que aparezcan más de 500 veces
frequent_products = order_products_prior["product_id"].value_counts()
popular_products = frequent_products[frequent_products > 500].index

# 2. Filtrar el dataframe original
filtered_op_prior = order_products_prior[order_products_prior["product_id"].isin(popular_products)]

# 3. Tomar una muestra de pedidos (por ejemplo, 100.000 pedidos distintos)
sampled_order_ids = filtered_op_prior["order_id"].drop_duplicates().sample(n=100_000, random_state=42)

# 4. Crear la lista de transacciones (listas de productos por pedido)
basket = filtered_op_prior[filtered_op_prior["order_id"].isin(sampled_order_ids)] \
    .groupby("order_id")["product_id"].apply(list)

transactions = basket.tolist()


Paso 2: Convertir a formato one-hot (sparse para ahorrar RAM)

In [22]:
from mlxtend.preprocessing import TransactionEncoder

# Codificación one-hot (ahorro de memoria usando sparse=True)
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions, sparse=True)

# Convertir a dataframe sparse
import pandas as pd
df_transactions = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


/tmp/ipykernel_7667/3804387274.py:9: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df_transactions = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


Paso 3: Aplicar Apriori para encontrar itemsets frecuentes

In [ ]:
from mlxtend.frequent_patterns import apriori

# Convertir nombres de columnas a string (requerido por mlxtend con sparse)
df_transactions.columns = df_transactions.columns.astype(str)

# Extraer itemsets frecuentes con soporte mínimo (ajustable)
frequent_itemsets = apriori(df_transactions, min_support=0.005, use_colnames=True)

# Mostrar los más frecuentes
print(frequent_itemsets.sort_values("support", ascending=False).head(10))


    support itemsets
42  0.15059  (24852)
21  0.11865  (13176)
31  0.08287  (21137)
34  0.07512  (21903)
96  0.06697  (47209)
98  0.05497  (47766)
97  0.04720  (47626)
45  0.04472  (26209)
52  0.04458  (27845)
24  0.04451  (16797)


Paso 4: Generar reglas de asociación

In [27]:
from mlxtend.frequent_patterns import association_rules

# Crear reglas con confianza mínima del 30%
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

# Ordenar por lift (las más interesantes)
rules = rules.sort_values("lift", ascending=False)

# Ver las mejores 10 reglas
print(rules[["antecedents", "consequents", "support", "confidence", "lift"]].head(10))


  antecedents consequents  support  confidence      lift
0     (28204)     (24852)  0.01009    0.375372  2.492676
1     (47766)     (24852)  0.01653    0.300709  1.996875


In [28]:
# Convertimos las reglas a un formato fácil de consultar
# Por cada antecedente, nos quedamos con el consecuente más fuerte
top_rules = rules.sort_values("lift", ascending=False).drop_duplicates(subset="antecedents")

# Creamos un diccionario: antecedente → consecuente
reglas_dict = {tuple(ant): list(cons)[0] for ant, cons in zip(top_rules["antecedents"], top_rules["consequents"])}

# Función para recomendar productos en base a lo que ha comprado un usuario
def recomendar_mba(user_id, ordenes_df, reglas_dict):
    productos_usuario = ordenes_df[ordenes_df["user_id"] == user_id]["product_id"].unique()
    recomendaciones = set()
    for producto in productos_usuario:
        if (producto,) in reglas_dict:
            recomendaciones.add(reglas_dict[(producto,)])
    return list(recomendaciones)

# Ejemplo: recomendaciones MBA para el usuario 1
recomendaciones_mba = recomendar_mba(1, order_prior_merged, reglas_dict)

# Mostrar con nombres
productos_mba = products[products["product_id"].isin(recomendaciones_mba)]
print(productos_mba[["product_id", "product_name"]])


Empty DataFrame
Columns: [product_id, product_name]
Index: []
